##### Copyright 2019 The TensorFlow Authors.

In [94]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# CycleGAN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/cyclegan"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/generative/cyclegan.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/cyclegan.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/generative/cyclegan.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This notebook demonstrates unpaired image to image translation using conditional GAN's, as described in [Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks](https://arxiv.org/abs/1703.10593), also known as CycleGAN. The paper proposes a method that can capture the characteristics of one image domain and figure out how these characteristics could be translated into another image domain, all in the absence of any paired training examples. 

This notebook assumes you are familiar with Pix2Pix, which you can learn about in the [Pix2Pix tutorial](https://www.tensorflow.org/tutorials/generative/pix2pix). The code for CycleGAN is similar, the main difference is an additional loss function, and the use of unpaired training data.

CycleGAN uses a cycle consistency loss to enable training without the need for paired data. In other words, it can translate from one domain to another without a one-to-one mapping between the source and target domain. 

This opens up the possibility to do a lot of interesting tasks like photo-enhancement, image colorization, style transfer, etc. All you need is the source and the target dataset (which is simply a directory of images).

![Output Image 1](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/horse2zebra_1.png?raw=1)
![Output Image 2](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/horse2zebra_2.png?raw=1)

## Set up the input pipeline

Install the [tensorflow_examples](https://github.com/tensorflow/examples) package that enables importing of the generator and the discriminator.

In [95]:
!pip install git+https://github.com/tensorflow/examples.git

  Cloning https://github.com/tensorflow/examples.git to c:\users\mtapi\appdata\local\temp\pip-req-build-31qqsnns
  Resolved https://github.com/tensorflow/examples.git to commit bf193f680a7d7ea25cd76046c10dac0335696255
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for tensorflow-examples: filename=tensorflow_examples-bf193f680a7d7ea25cd76046c10dac0335696255_-py3-none-any.whl size=301320 sha256=1d7876035dc9c024516273e80b2caffffe2abb958bb098eff0c727103116bdb9
  Stored in directory: C:\Users\mtapi\AppData\Local\Temp\pip-ephem-wheel-cache-_ur0hf4o\wheels\72\5f\d0\7fe769eaa229bf20101d11a357eb23c83c481bee2d7f710599
Failed to build tensorflow-examples
  Attempting uninstall: tensorflow-examples
    Found existing installation: tensorflow-examples e04f01490583a53f220e24990b4cdbc9393ce006-
    Uninstalling tensorflow-examples-e04f01490583a53f220e24990b4cdbc9393ce006-:
      Successfully uninstalled tensorflow-examples-e04f01

  Running command git clone --filter=blob:none --quiet https://github.com/tensorflow/examples.git 'C:\Users\mtapi\AppData\Local\Temp\pip-req-build-31qqsnns'
  DEPRECATION: tensorflow-examples was installed using the legacy 'setup.py install' method, because a wheel could not be built for it. A possible replacement is to fix the wheel build issue reported above. Discussion can be found at https://github.com/pypa/pip/issues/8368


In [96]:
import tensorflow as tf

In [97]:
import glob
from PIL import Image

In [98]:
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix

import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import pandas as pd

AUTOTUNE = tf.data.AUTOTUNE

## Input Pipeline

This tutorial trains a model to translate from images of horses, to images of zebras. You can find this dataset and similar ones [here](https://www.tensorflow.org/datasets/catalog/cycle_gan). 

As mentioned in the [paper](https://arxiv.org/abs/1703.10593), apply random jittering and mirroring to the training dataset. These are some of the image augmentation techniques that avoids overfitting.

This is similar to what was done in [pix2pix](https://www.tensorflow.org/tutorials/generative/pix2pix#load_the_dataset)

* In random jittering, the image is resized to `286 x 286` and then randomly cropped to `256 x 256`.
* In random mirroring, the image is randomly flipped horizontally i.e left to right.

In [99]:
#from google.colab import drive
#drive_dir = '/content/drive'
#drive.mount(drive_dir, force_remount=True) 

In [100]:
main_dir = f'./'
landscape_dir = 'Landscape_Images' 
camo_dir = 'camo_subset_raw'

landscape_path = f"{main_dir}/{landscape_dir}"
camo_path = f"{main_dir}/{camo_dir}"

print(landscape_path, camo_path)

.//Landscape_Images .//camo_subset_raw


In [101]:
# file_path = landscape_path

# images_dict = {os.path.splitext(os.path.basename(x))[0]: x
#                     for x in glob.glob(os.path.join(file_path, '', '*.jpg'))}

In [102]:
# df = pd.DataFrame(images_dict.items())
# #rename columns for clarity
# df = df.rename(columns={df.columns[0]: "image_id"})
# df = df.rename(columns={df.columns[1]: "image_path"})

# df["image"] = df['image_path'].map(lambda x: np.asarray(Image.open(x).resize((256, 256))))

# imgs = df["image"]
# inputs = np.array(imgs)/(255.0) # normalization

# interim_inputs = [j for i, j in enumerate(inputs)]

In [103]:
# print([i.shape for i in interim_inputs])

In [104]:
# dataset, metadata = tfds.load('cycle_gan/horse2zebra',
#                               with_info=True, as_supervised=True)

# train_horses, train_zebras = dataset['trainA'], dataset['trainB']
# test_horses, test_zebras = dataset['testA'], dataset['testB']
def data_preprocessor(file_path, limit=None):

    #create dataframe to store image paths and images:

    images_dict = {os.path.splitext(os.path.basename(x))[0]: x
                    for x in glob.glob(os.path.join(file_path, '', '*.jpg'))}

    df = pd.DataFrame(images_dict.items())
    #rename columns for clarity
    df = df.rename(columns={df.columns[0]: "image_id"})
    df = df.rename(columns={df.columns[1]: "image_path"})
    if limit:
      df = df[:limit]
    df["image"] = df['image_path'].map(lambda x: np.asarray(Image.open(x).resize((256, 256))))

    imgs = df["image"]
    inputs = np.array(imgs)/(255.0) # normalization

    interim_inputs = [j for i, j in enumerate(inputs)]
    inp_reshape = tf.reshape(interim_inputs, (-1, 256, 256, 3))
    train_imgs = np.asarray(inp_reshape, dtype= np.float32)

    # train_imgs = tf.random.shuffle(final_inputs)

    og_images = []
    for tensor in train_imgs:
        og_images.append(tensor)

    # Apply data augmentation to populate some data
    # With data augmentation to prevent overfitting
    """
    lst_saturated = []
    for i in range(len(train_imgs)):
        saturation_played_1_3 = tf.image.adjust_saturation(train_imgs[i], 1.3)
        saturation_played_1_6 = tf.image.adjust_saturation(train_imgs[i], 1.6)
        saturation_played_1_9 = tf.image.adjust_saturation(train_imgs[i], 1.9)
        lst_saturated.append(saturation_played_1_3)
        lst_saturated.append(saturation_played_1_6)
        lst_saturated.append(saturation_played_1_9)

    res_list = [y for x in [og_images, lst_saturated] for y in x]
    """

    tensor_converted_images = tf.convert_to_tensor(og_images)
    image_dataset = tf.data.Dataset.from_tensor_slices(tensor_converted_images)

    ds_size = tf.data.experimental.cardinality(image_dataset)
    train_split=0.8
    test_split=0.2
    shuffle_size=296

    Shuffle=True
    if Shuffle:
    # Specify seed to always have the same split distribution between runs
        ds = image_dataset.shuffle(shuffle_size, seed=12)

    train_size = int(np.ceil(train_split * int(ds_size)))
    test_size = int(np.ceil(test_split * int(ds_size)))

    train_ds = ds.take(train_size)   
    test_ds = ds.take(test_size)

    train_size_lst = []
    for img in train_ds:
        train_size_lst.append(img)

    train_imgs_arrays = []
    for tensor_images in train_size_lst:
        array_img = np.asarray(tensor_images)
        train_imgs_arrays.append(array_img)

    # dark_skin_sample = train_imgs_arrays[20]

    return train_ds, test_ds # , dark_skin_sample


In [105]:
LIMIT = 300

train_landscape, test_landscape = data_preprocessor(landscape_path, limit=LIMIT)
train_camo, test_camo = data_preprocessor(camo_path, limit=LIMIT)

KeyboardInterrupt: 

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
def random_crop(image):
  cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image

In [ ]:
# normalizing the images to [-1, 1]
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

In [ ]:
def random_jitter(image):
  # resizing to 286 x 286 x 3
  image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # randomly cropping to 256 x 256 x 3
  image = random_crop(image)

  # random mirroring
  image = tf.image.random_flip_left_right(image)

  return image

In [ ]:
def preprocess_image_train(image, label):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [ ]:
def preprocess_image_test(image, label):
  image = normalize(image)
  return image

In [ ]:
# train_horses = train_horses.cache().map(
#     preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
#     BUFFER_SIZE).batch(BATCH_SIZE)

# train_zebras = train_zebras.cache().map(
#     preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
#     BUFFER_SIZE).batch(BATCH_SIZE)

# test_horses = test_horses.map(
#     preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
#     BUFFER_SIZE).batch(BATCH_SIZE)

# test_zebras = test_zebras.map(
#     preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
#     BUFFER_SIZE).batch(BATCH_SIZE)

train_landscapes = train_landscape.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_landscapes = test_landscape.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_camos = train_camo.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_camos = test_camo.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

## Sample Landscape and Camo for benchmarks/vis

In [ ]:
sample_landscape = next(iter(train_landscapes))
sample_camo = next(iter(train_camos))

## Template for displaying images (generated or otherwise)

In [ ]:
plt.subplot(121)
plt.title('Landscape')
plt.imshow(sample_landscape[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Landscape with random jitter')
plt.imshow(random_jitter(sample_landscape[0]) * 0.5 + 0.5)

In [ ]:
plt.subplot(121)
plt.title('Camo')
plt.imshow(sample_camo[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Camo with random jitter')
plt.imshow(random_jitter(sample_camo[0]) * 0.5 + 0.5)

## Import and reuse the Pix2Pix models

Import the generator and the discriminator used in [Pix2Pix](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py) via the installed [tensorflow_examples](https://github.com/tensorflow/examples) package.

The model architecture used in this tutorial is very similar to what was used in [pix2pix](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py). Some of the differences are:

* Cyclegan uses [instance normalization](https://arxiv.org/abs/1607.08022) instead of [batch normalization](https://arxiv.org/abs/1502.03167).
* The [CycleGAN paper](https://arxiv.org/abs/1703.10593) uses a modified `resnet` based generator. This tutorial is using a modified `unet` generator for simplicity.

There are 2 generators (G and F) and 2 discriminators (X and Y) being trained here. 

* Generator `G` learns to transform image `X` to image `Y`. $(G: X -> Y)$
* Generator `F` learns to transform image `Y` to image `X`. $(F: Y -> X)$
* Discriminator `D_X` learns to differentiate between image `X` and generated image `X` (`F(Y)`).
* Discriminator `D_Y` learns to differentiate between image `Y` and generated image `Y` (`G(X)`).

![Cyclegan model](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/cyclegan_model.png?raw=1)

In [ ]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [ ]:
to_camo = generator_g(sample_landscape)
to_landscape = generator_f(sample_camo)
plt.figure(figsize=(8, 8))
contrast = 8

imgs = [sample_landscape, to_camo, sample_camo, to_landscape]
title = ['Landscape', 'To Camo', 'Camo', 'To Landscape']

for i in range(len(imgs)):
  plt.subplot(2, 2, i+1)
  plt.title(title[i])
  if i % 2 == 0:
    plt.imshow(imgs[i][0] * 0.5 + 0.5)
  else:
    plt.imshow(imgs[i][0] * 0.5 * contrast + 0.5)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(121)
plt.title('Is a real camo?')
plt.imshow(discriminator_y(sample_camo)[0, ..., -1], cmap='RdBu_r')

plt.subplot(122)
plt.title('Is a real landscape?')
plt.imshow(discriminator_x(sample_landscape)[0, ..., -1], cmap='RdBu_r')

plt.show()

## Loss functions

In CycleGAN, there is no paired data to train on, hence there is no guarantee that the input `x` and the target `y` pair are meaningful during training. Thus in order to enforce that the network learns the correct mapping, the authors propose the cycle consistency loss.

The discriminator loss and the generator loss are similar to the ones used in [pix2pix](https://www.tensorflow.org/tutorials/generative/pix2pix#build_the_generator).

In [ ]:
LAMBDA = 1

In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

In [ ]:
def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

Cycle consistency means the result should be close to the original input. For example, if one translates a sentence from English to French, and then translates it back from French to English, then the resulting sentence should be the same as the  original sentence.

In cycle consistency loss, 

* Image $X$ is passed via generator $G$ that yields generated image $\hat{Y}$.
* Generated image $\hat{Y}$ is passed via generator $F$ that yields cycled image $\hat{X}$.
* Mean absolute error is calculated between $X$ and $\hat{X}$.

$$forward\ cycle\ consistency\ loss: X -> G(X) -> F(G(X)) \sim \hat{X}$$

$$backward\ cycle\ consistency\ loss: Y -> F(Y) -> G(F(Y)) \sim \hat{Y}$$


![Cycle loss](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/cycle_loss.png?raw=1)

In [ ]:
def calc_cycle_loss(real_image, cycled_image, loss_weight=10):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
  return loss_weight * loss1

As shown above, generator $G$ is responsible for translating image $X$ to image $Y$. Identity loss says that, if you fed image $Y$ to generator $G$, it should yield the real image $Y$ or something close to image $Y$.

If you run the zebra-to-horse model on a horse or the horse-to-zebra model on a zebra, it should not modify the image much since the image already contains the target class.

$$Identity\ loss = |G(Y) - Y| + |F(X) - X|$$

In [ ]:
def identity_loss(real_image, same_image, loss_wieght=10):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return loss_wieght * 0.5 * loss

Initialize the optimizers for all the generators and the discriminators.

In [ ]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

## Checkpoints

In [ ]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

## Training

Note: This example model is trained for fewer epochs (40) than the paper (200) to keep training time reasonable for this tutorial. Predictions may be less accurate. 

In [ ]:
EPOCHS = 40

In [ ]:
def generate_images(model, test_input):
  prediction = model(test_input)
    
  plt.figure(figsize=(12, 12))

  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [ ]:
def generate_patched_landscape(model, landscape):
    fake_camo = generator_g(landscape)
    patched_landscape = cutout_and_replace(sample_landscape, fake_camo, cutout_size=64)

    plt.figure(figsize=(12, 12))

    display_list = [landscape[0], patched_landscape[0]]
    title = ['Input Landscape', 'Patched Landscape']

    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.show()

In [ ]:
def save_generated_results(model, landscape, filename):
    fake_camo = model(landscape)
    patched_landscape = cutout_and_replace(landscape, fake_camo, cutout_size=64)

    plt.figure(figsize=(12, 12))

    display_list = [fake_camo[0], patched_landscape[0]]
    title = ['Fake Camo', 'Patched Landscape']

    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.title(title[i])
        # getting the pixel values between [0, 1] to plot it.
        plt.imshow(display_list[i] * 0.5 + 0.5)
        plt.axis('off')
    plt.savefig(filename)

Even though the training loop looks complicated, it consists of four basic steps:

* Get the predictions.
* Calculate the loss.
* Calculate the gradients using backpropagation.
* Apply the gradients to the optimizer.

In [ ]:
def cutout_and_replace(orig_landscape, camo, cutout_size=16):
    H, W = orig_landscape.shape[1:3]
    assert cutout_size <= H//2

    # Determining square location
    H_start = int(H//2)
    x_start = np.random.randint(0, W - cutout_size)
    y_start = np.random.randint(H_start, H - cutout_size)

    # Square indices
    xs = list(range(x_start, x_start+cutout_size))
    ys = list(range(y_start, y_start+cutout_size))
    list_idxs = np.dstack(np.meshgrid(ys, xs)).reshape(-1, 2)

    # Resizing camo patch
    resized_camo_patch = tf.image.resize(camo, [cutout_size, cutout_size])
    resized_camo_patch = tf.transpose(resized_camo_patch, perm=[0, 2, 1, 3]) # Transpose image
    list_camo_pix = tf.reshape(resized_camo_patch, [cutout_size**2, 3])

    # Replacing with resized camo
    updated_landscape = tf.tensor_scatter_nd_update(orig_landscape[0], list_idxs, list_camo_pix)
    print(updated_landscape.shape)

    return tf.expand_dims(updated_landscape, 0) # Convert back to (1, H, W, 3)


In [ ]:
generate_patched_landscape(generator_g, sample_landscape)

In [ ]:
@tf.function
def train_step(real_x, real_y, reconst_weight=10, ls_disc_weight=1):
  # y - camo
  # x - landscape
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.
    
    fake_y = generator_g(real_x, training=True) # generated camouflage
    # cycled_x = generator_f(fake_y, training=True)

    # fake_x = generator_f(real_y, training=True)
    fake_x = cutout_and_replace(real_x, fake_y, cutout_size=64) # cutout/replaced landscape
    cycled_y = generator_g(fake_x, training=True) # Might not be necessary for camo GAN; how close does the generator get to producing the same camouflage from landscapes patched by camouflage?

    # same_x and same_y are used for identity loss.
    # same_x = generator_f(real_x, training=True)
    same_y = generator_g(real_y, training=True) # Can generator G produce the same camouflage given itself? (identity)

    disc_real_x = discriminator_x(real_x, training=True) # Is real landscape a landscape?
    disc_real_y = discriminator_y(real_y, training=True) # Is real camouflage a camouflage?

    disc_fake_x = discriminator_x(fake_x, training=True) # Tests: is cutout/replaced landscape a landscape?
    disc_fake_y = discriminator_y(fake_y, training=True) # Tests: is generated camouflage a camouflage?

    # calculate the loss
    gen_g_loss = generator_loss(disc_fake_y)+ ls_disc_weight*generator_loss(disc_fake_x) # Want to fool both discriminators 
    # gen_f_loss = generator_loss(disc_fake_x)
    
    # total_cycle_loss = calc_cycle_loss(real_x, cycled_x, reconst_weight) + calc_cycle_loss(real_y, cycled_y, reconst_weight)
    total_cycle_loss = calc_cycle_loss(real_y, cycled_y, reconst_weight)
    
    # Total generator loss = adversarial loss + cycle loss
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y, reconst_weight)
    # total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x, reconst_weight)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
  # Calculate the gradients for generator and discriminator
  generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
  # generator_f_gradients = tape.gradient(total_gen_f_loss, 
  #                                       generator_f.trainable_variables)
  
  discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)
  
  # Apply the gradients to the optimizer
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

  # generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
  #                                           generator_f.trainable_variables))
  
  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))
  
  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

## File path to save images:

In [ ]:
image_path = f'{main_dir}/images'
print(image_path)

## Patched-Camo GAN:

reconst_weight=0.5, ls_disc_weight=1

In [ ]:
reconst_weight=0.5
ls_disc_weight=1

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))
                                                      

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

### reconst_weight=0.5, ls_disc_weight=2

In [ ]:
reconst_weight=0.5
ls_disc_weight=2

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

### reconst_weight=0.5, ls_disc_weight=5

In [ ]:
reconst_weight=0.5
ls_disc_weight=5

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

## reconst_weight=0.5, ls_disc_weight=10

In [ ]:
reconst_weight=0.5
ls_disc_weight=10

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

## reconst_weight=0, ls_disc_weight=1

In [ ]:
reconst_weight=0
ls_disc_weight=1

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

## reconst_weight=0, ls_disc_weight=2

In [ ]:
reconst_weight=0
ls_disc_weight=2

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

## reconst_weight=0, ls_disc_weight=5

In [ ]:
reconst_weight=0
ls_disc_weight=5

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

## reconst_weight=0, ls_disc_weight=10

In [ ]:
reconst_weight=0
ls_disc_weight=10

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

## reconst_weight=5, ls_disc_weight=1

In [ ]:
reconst_weight=5
ls_disc_weight=1

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

## reconst_weight=5, ls_disc_weight=2

In [ ]:
reconst_weight=5
ls_disc_weight=2

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

## reconst_weight=5, ls_disc_weight=5

In [ ]:
reconst_weight=5
ls_disc_weight=5

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

In [ ]:
# Reset models
generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

## reconst_weight=5, ls_disc_weight=10

In [ ]:
reconst_weight=5
ls_disc_weight=10

for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=reconst_weight, ls_disc_weight=ls_disc_weight)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)
  generate_patched_landscape(generator_g, sample_landscape)


  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    # Save generated results:
    print("SAVING RESULTS")
    for i, test_ls in enumerate(test_landscapes.take(10)):
      fc_pl_filename = f'{image_path}/reconst_{reconst_weight}_lsdisc_{ls_disc_weight}_fc_pl_epoch_{epoch+1}_{i}.png'

      save_generated_results(generator_g, test_ls, fc_pl_filename)
      

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

# OLD Experiments:

## LAMBDA = 10

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

## Generate using test dataset

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

# Model with LAMBDA = 0.5

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=0.5)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

# Lambda = 0.1

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=0.1)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

# Lambda = 0

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_landscapes, train_camos)):
    train_step(image_x, image_y, reconst_weight=0)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_landscape)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_landscapes.take(10):
  generate_images(generator_g, inp)

# Camo to Landscape:

Reconst weight 10

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_camos, train_landscapes)):
    train_step(image_x, image_y)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_camo)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_camos.take(10):
  generate_images(generator_g, inp)

Reconst weight 1

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_camos, train_landscapes)):
    train_step(image_x, image_y, reconst_weight=1)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_camo)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_camos.take(10):
  generate_images(generator_g, inp)

Reconst Weight 0

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_camos, train_landscapes)):
    train_step(image_x, image_y, reconst_weight=0)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_camo)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

In [ ]:
# Run the trained model on the test dataset
for inp in test_camos.take(10):
  generate_images(generator_g, inp)

## Next steps

This tutorial has shown how to implement CycleGAN starting from the generator and discriminator implemented in the [Pix2Pix](https://www.tensorflow.org/tutorials/generative/pix2pix) tutorial. As a next step, you could try using a different dataset from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/cycle_gan). 

You could also train for a larger number of epochs to improve the results, or you could implement the modified ResNet generator used in the [paper](https://arxiv.org/abs/1703.10593) instead of the U-Net generator used here.

-----------------------------------

# Experiments/Scratchwork

## Determining cutout location

In [ ]:
cutout_size=100

H, W = sample_landscape[0].shape[0:2]
H_start = int(H//2)
x_start = np.random.randint(0, W - cutout_size)
y_start = np.random.randint(H_start, H - cutout_size)

print(x_start, y_start)

## Getting indices of cutout square

In [ ]:
xs = list(range(x_start, x_start+cutout_size))
ys = list(range(y_start, y_start+cutout_size))

pairs = np.dstack(np.meshgrid(xs, ys)).reshape(-1, 2)

In [ ]:
# top_left_x = x_start
# top_left_y = y_start
# top_left_z = 0
# roi_len_x = cutout_size
# roi_len_y = cutout_size
# bottom_right_z = 3

# roi_slice = tf.slice(
#   sample_landscape[0],
#   [top_left_x, top_left_y, top_left_z],
#   [roi_len_x, roi_len_y, bottom_right_z]
# )
# print(roi_slice.shape)
# print(roi_slice)

In [ ]:
# roi_mask = tf.ones_like(roi_slice)
# mask_canvas = tf.image.pad_to_bounding_box(
#   [roi_mask],
#   top_left_x,
#   top_left_y,
#   H,
#   W
# )
# bool_mask = tf.cast(mask_canvas, tf.bool)
# print(bool_mask)

In [ ]:




resized_camo_patch = tf.image.resize(sample_camo, [cutout_size, cutout_size])

# # Make an editable copy of the image
# editable_image = tf.Variable(initial_value=sample_landscape, dtype=tf.float32)
# # init_op = tf.assign(editable_image, sample_landscape)

# # Make sure we don't update the image before we've set its initial value.
# # with tf.control_dependencies([init_op]):
# update_roi.assign(editable_image, resized_camo_patch)


# # indices = tf.constant(bool_mask)

# # update = tf.tensor_scatter_nd_update(sample_landscape, indices, resized_camo_patch)

In [ ]:
# Only want x and y, from (1, 16, 16, 3)
# np_bool_mask = np.array(np.where(bool_mask[0,:,:,0]))
# print(np_bool_mask.shape)

# list_bool_idxs = np_bool_mask.transpose()
list_bool_idxs = pairs
print(list_bool_idxs.shape)

list_camo_pix = tf.reshape(resized_camo_patch, [cutout_size**2, 3])
print(list_camo_pix.shape)

In [ ]:
# Ex. list of shape 4x2, want to input a channel(3)-size update
# indices = [[0, 0], [1, 1], [2, 2], [3, 3]]
# updates = tf.ones([4, 3])

resized_camo_patch = tf.image.resize(sample_camo, [cutout_size, cutout_size])
indices = tf.constant(list_bool_idxs)
updates = tf.constant(list_camo_pix)
# updates = tf.zeros([cutout_size**2, 3])
print(resized_camo_patch.shape)
print(indices.shape)
print(updates.shape)

update = tf.tensor_scatter_nd_update(sample_landscape[0], list_bool_idxs, list_camo_pix)
print(update.shape)

update = cutout_and_replace(sample_landscape, sample_camo, cutout_size=100)

plt.subplot(121)
plt.title('Landscape')
plt.imshow(update * 0.5 + 0.5)